# Movies 3

## Introduction

Une base de données (hébergée dans le cloud AWS) contenant un certain nombre de données cinématographiques vous est fournie pour exploration et mise en pratique des langages SQL et Python.  
Les données de connexion (user, host) sont indiquées dans le brief, le mot de passe nécessaire est précisé en session par le formateur.  


## 1. Description Générale

En utilisant DBeaver et les paramètres de connexion donnés par ailleurs, faire une description détaillée de la base de données fournie. Présenter notamment le diagramme entité - relation (ERD) et les différentes caractéristiques des tables de la base.  Quelles types de renseignement peut-on tirer de cette base de données ?   

---

## 2. Exploration SQL via python / pandas
Dans cette partie, on s'attache à explorer la base de données en utilisant le language SQL pour les requêtes et le language Python (+ module pandas) pour récupérer les résultats de façon programmatique.  
Vous pouvez utiliser DBeaver ou outil similaire pour préparer / tester vos requêtes mais celles-ci doivent être reportées et appliquées dans du code Python comme présenté dans les quelques examples à suivre.  
*Pour les résultats "longs", un aperçu des quelques premières lignes et le comptage total des résultats est suffisant.*

In [1]:
import os
import pandas as pd

In [2]:
from sqlalchemy import create_engine
import pymysql

In [3]:
# paramètres de connexion à la base de données movies (MySQL sous AWS RDS)
user = 'student'
password = 'dev-data-nte-2021'
host = 'simplon-mysql.cpreezidasug.eu-west-1.rds.amazonaws.com'
port = 3306
dbname = 'movies'  

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}")

### Exemples

#### Tables de base

In [4]:
movies = pd.read_sql_query("SELECT * FROM movies LIMIT 10", engine)
movies.head()

,id,title,year,directorId,genre,summary,countrycode
0,11,La Guerre des étoiles,1977,1,Aventure,"Il y a bien longtemps, dans une galaxie très l...",US
1,24,Kill Bill : Volume 1,2003,138,Action,Au cours d'une cérémonie de mariage en plein d...,US
2,28,Apocalypse Now,1979,1776,Drame,L'état-major américain confie au jeune capitai...,US
3,33,Impitoyable,1992,190,Western,"Après avoir été un impitoyable tueur, toujours...",US
4,38,Eternal Sunshine of the Spotless Mind,2004,201,Science-Fiction,Joël et Clémentine ne voient plus que les mauv...,US


In [5]:
artists = pd.read_sql_query("SELECT * FROM artists LIMIT 10", engine)
artists.head()

,id,lastname,firstname,birthyear
0,1,Lucas,George,1944
1,2,Hamill,Mark,1951
2,3,Ford,Harrison,1942
3,4,Fisher,Carrie,1956
4,5,Cushing,Peter,1913


In [6]:
roles = pd.read_sql_query("SELECT * FROM roles LIMIT 10", engine)
roles.head()

,movieId,actorId,character
0,11,2,Luke Skywalker
1,1891,2,Luke Skywalker
2,1892,2,Luke Skywalker
3,181808,2,Luke Skywalker / Dobbu Scay (voice)
4,11,3,Han Solo


In [7]:
countries = pd.read_sql_query("SELECT * FROM countries LIMIT 10", engine)
countries.head()

,code,name,language
0,AU,Australia,
1,BE,Belgium,
2,BS,Bahamas,
3,CA,Canada,
4,CN,China,


#### Autre requête simple
Q00: Nom et année de naissance des artistes nés après 1975 ? Combien sont-ils ?

In [8]:
sql_query_00 = """
    SELECT *
    FROM artists
    WHERE birthyear > 1975
"""

results = pd.read_sql_query(sql_query_00, engine)
results.head(5)

,id,lastname,firstname,birthyear
0,131,Gyllenhaal,Jake,1980
1,205,Dunst,Kirsten,1982
2,225,Holmes,Ashton,1978
3,258,García Bernal,Gael,1978
4,368,Witherspoon,Reese,1976


In [9]:
results.shape

(118, 4)

In [10]:
len(results)

118

### Questions

Q01: Quels sont les films de type 'Drame' ? Combien cela représente de films ?  

In [11]:
drame = pd.read_sql_query("SELECT title FROM movies WHERE genre='Drame'", engine)
drame

,title
0,Apocalypse Now
1,A History of Violence
2,Match point
3,Le Secret de Brokeback Mountain
4,Breaking the Waves
...,...
78,Une vie violente
79,En guerre
80,Amanda
81,Green Book : Sur les routes du sud


Q02: Quels rôles a joué Bruce Willis et dans quels films (triés par ordre chronologique) ?

In [12]:
bruce = pd.read_sql_query("SELECT id FROM artists WHERE lastname='Willis' and firstname='Bruce'", engine)
films = pd.read_sql_query(f"SELECT movieId, `character`, title FROM roles INNER JOIN movies ON roles.movieId = movies.id WHERE actorId={bruce.id[0]}", engine)
films

,movieId,character,title
0,562,John McClane,Piège de cristal
1,680,Butch Coolidge,Pulp Fiction
2,1571,John McClane,Die Hard 4 : Retour en enfer
3,1572,John McClane,Une Journée en enfer
4,1573,John McClane,58 minutes pour vivre


Q03: Qui est le réalisateur de Memento ?

In [13]:
memento_real = pd.read_sql_query("SELECT directorId, lastname, firstname FROM movies INNER JOIN artists ON movies.directorId = artists.id WHERE title='Memento'", engine)
memento_real

,directorId,lastname,firstname
0,525,Nolan,Christopher


Q04: Qui a joué le rôle de Chewbacca et dans quels films ?

In [14]:
chewbacca = pd.read_sql_query("SELECT lastname, firstname, id FROM roles INNER JOIN artists ON roles.actorId = artists.id WHERE `character` = 'Chewbacca'", engine)
chewbacca

,lastname,firstname,id
0,Mayhew,Peter,24343
1,Mayhew,Peter,24343
2,Mayhew,Peter,24343
3,Mayhew,Peter,24343
4,Suotamo,Joonas,1709041


Q05: Dans quels films Bruce Willis a-t-il joué le role de John McClane ? 


In [15]:
#films = pd.read_sql_query(f"SELECT movieId, `character`, title, FROM roles INNER JOIN movies ON roles.movieId = movies.id WHERE `character` = 'John Mclane'", engine)
bruce = pd.read_sql_query("SELECT title FROM roles INNER JOIN movies ON roles.movieId = movies.id WHERE `character` = 'John McClane'", engine)
bruce

,title
0,Piège de cristal
1,Die Hard 4 : Retour en enfer
2,Une Journée en enfer
3,58 minutes pour vivre


Q06: Quels sont les acteurs de 'Sueurs froides' ?

In [16]:
movie = pd.read_sql_query("SELECT * FROM movies WHERE title='Sueurs froides'", engine)
#actors = pd.read_sql_query("", engine)

pd.read_sql_query(f"SELECT * FROM roles WHERE movieId ={movie.id[0]}", engine)

,movieId,actorId,character
0,426,854,Det. John 'Scottie' Ferguson
1,426,5729,Madeleine Elster / Judy Barton
2,426,5730,Marjorie 'Midge' Wood


Q07: Quels sont les films dont le réalisateur est Tim Burton, et l’un des acteurs Jonnhy Depp ?

In [17]:
tim = pd.read_sql_query("SELECT id FROM artists WHERE lastname='Burton' and firstname='Tim'", engine)
johnny = pd.read_sql_query("SELECT id FROM artists WHERE lastname='Depp' and firstname='Johnny'", engine)
res = pd.read_sql_query(f"SELECT movieId, title FROM roles INNER JOIN movies ON roles.movieId = movies.id WHERE actorId = {johnny.id[0]} AND directorId = {tim.id[0]}", engine)
res

,movieId,title
0,2668,"Sleepy Hollow, La Légende du cavalier sans tête"


Q08: Quels sont les films dans lesquels a joué Woody Allen ? Donnez aussi le rôle.

In [18]:
woody = pd.read_sql_query("SELECT id FROM artists WHERE lastname='Allen' and firstname='Woody'", engine)
res = pd.read_sql_query(f"SELECT title, `character` FROM roles INNER JOIN movies ON roles.movieid = movies.id WHERE actorId = {woody.id[0]}", engine)
res

,title,character
0,Scoop,Sid Waterman
1,Manhattan,Isaac Davis
2,Annie Hall,Alvy Singer
3,Maris et femmes,Prof. Gabriel 'Gabe' Roth


Q09: Quel metteur en scène a tourné dans ses propres films ? Donnez le nom, le rôle et le titre des films en question.

In [19]:
res = pd.read_sql_query("SELECT title, lastname, firstname FROM movies INNER JOIN artists ON movies.directorId = artists.id WHERE EXISTS (SELECT * FROM roles WHERE movies.directorId = roles.actorId)", engine)
res

,title,lastname,firstname
0,Kill Bill : Volume 1,Tarantino,Quentin
1,Impitoyable,Eastwood,Clint
2,Match point,Allen,Woody
3,Les Quatre Cents Coups,Truffaut,François
4,Lost in Translation,Coppola,Sofia
5,Jackie Brown,Tarantino,Quentin
6,Kill Bill : Volume 2,Tarantino,Quentin
7,Le Pianiste,Polanski,Roman
8,Reservoir Dogs,Tarantino,Quentin
9,Scoop,Allen,Woody


Q10: Quels sont les films de Quentin Tarantino dans lesquels il n’a pas joué 

In [20]:
pd.read_sql_query("SELECT title FROM movies WHERE directorId = 138 AND directorId not in(SELECT actorId FROM roles WHERE movies.id = roles.movieId)", engine)

,title
0,Kill Bill : Volume 1
1,Jackie Brown
2,Kill Bill : Volume 2
3,Inglourious Basterds
4,Django Unchained


Q11: Quel metteur en scène a tourné en tant qu’acteur ? Donner le nom, le rôle et le titre des films dans lesquels cet artiste a joué.

In [21]:
res = pd.read_sql_query("SELECT title, directorId, `character` FROM movies INNER JOIN roles ON movies.id = roles.movieId WHERE directorId = roles.actorId", engine)
res

,title,directorId,character
0,Impitoyable,190,Bill Munny
1,Reservoir Dogs,138,Mr. Brown
2,Scoop,1243,Sid Waterman
3,Pulp Fiction,138,Jimmie Dimmick
4,Manhattan,1243,Isaac Davis
5,Annie Hall,1243,Alvy Singer
6,Chantons sous la pluie,13294,Don Lockwood
7,Le dictateur,13848,"Adenoid Hynkel, Dictator of Tomania / A Jewish..."
8,Le Bal des vampires,3556,"Alfred, Assistent des Professors"
9,Les temps modernes,13848,A factory worker


Q12: Quels sont les films de Hitchcock sans James Stewart

In [22]:
pd.read_sql_query("SELECT title FROM movies INNER JOIN artists ON movies.directorId = artists.id WHERE directorId = 2636 AND title NOT IN(SELECT title FROM movies INNER JOIN roles ON movies.id = roles.movieId WHERE actorId = 854)", engine)

,title
0,La Mort aux trousses
1,Rebecca
2,Les Enchaînés
3,Psychose
4,Les Oiseaux
5,L'Inconnu du Nord-Express
6,Soupçons


Q13: Quel est la filmographie de James Stewart ?

In [23]:
pd.read_sql_query("SELECT movieId, title FROM roles INNER JOIN movies ON roles.movieId = movies.id WHERE actorId = 854", engine)

,movieId,title
0,426,Sueurs froides
1,567,Fenêtre sur cour
2,574,L'homme qui en savait trop


Q14: Quels sont les films sans rôle (acteurs) ?

In [24]:
pd.read_sql("SELECT title FROM movies LEFT JOIN roles ON movies.id = roles.movieId WHERE roles.movieId IS NULL", engine)

,title
0,La Guerre des Mondes


Q15: Combien d'acteurs n’ont jamais réalisé de film ? 

In [25]:
pd.read_sql("SELECT directorId,firstname, lastname FROM artists LEFT JOIN movies ON artists.id = movies.directorId WHERE movies.directorId IS NULL ORDER BY directorId ASC", engine)

,directorId,firstname,lastname
0,None,Bourvil,
1,None,Terry-Thomas,
2,None,Vivica,A. Fox
3,None,John,Adames
4,None,Amy,Adams
...,...,...,...
982,None,Jonathan,Zaccaï
983,None,Billy,Zane
984,None,Fa,Zeng Li
985,None,Zhang,Ziyi


Q16: Donnez les noms et prénoms des 10 réalisateurs les plus prolifiques, ainsi que le nombre de films qu’ils ont tournés ?

In [26]:
pd.read_sql("SELECT directorId, firstname, lastname , count(movies.id) as nb FROM movies INNER JOIN artists ON movies.directorId = artists.id GROUP BY directorId ORDER BY nb DESC LIMIT 10", engine)

,directorId,firstname,lastname,nb
0,488,Steven,Spielberg,13
1,2636,Alfred,Hitchcock,10
2,1243,Woody,Allen,8
3,138,Quentin,Tarantino,7
4,240,Stanley,Kubrick,7
5,525,Christopher,Nolan,6
6,1776,Francis,Ford Coppola,6
7,578,Ridley,Scott,6
8,3556,Roman,Polanski,6
9,1,George,Lucas,5


Q17: Donnez les noms et prénoms des réalisateurs qui ont tourné au moins deux films. Combien sont-ils ?

In [34]:
res =pd.read_sql("SELECT directorId, firstname, lastname, COUNT(movies.id) AS nb FROM movies INNER JOIN artists ON movies.directorId = artists.id GROUP BY directorId HAVING nb >= 2 ORDER BY nb DESC", engine)
len(res)

46

Q18: Dans quels films le réalisateur a-t-il le même prénom que l’un des interprètes ? (titre, nom du réalisateur, nom de l’interprète). Le réalisateur et l’interprète ne doivent pas être la même personne.

Q19: Quels sont les 10 pays produisant le plus de films ?

In [30]:
pd.read_sql("SELECT countrycode, count(countrycode) as nb FROM movies GROUP BY countrycode ORDER BY nb desc", engine)


,countrycode,nb
0,US,131
1,FR,39
2,GB,21
3,CA,6
4,DE,5
5,IT,5
6,JP,4
7,AU,3
8,SE,3
9,BE,2


Q20: Quels sont les 5 acteurs ayant le plus joué dans des films français ?

In [29]:
pd.read_sql("SELECT actorId, COUNT(movieId) as nb FROM roles INNER JOIN movies ON roles.movieId = movies.id WHERE movies.countrycode = 'FR' GROUP BY actorId ORDER BY nb desc", engine)

,actorId,nb
0,3784,3
1,3829,3
2,50,3
3,11546,2
4,17498,2
...,...,...
194,2045568,1
195,15481,1
196,971262,1
197,1137823,1


---

## 3. Expression libre

A vous de jouer ! Tentez quelques requêtes de votre choix (avec pourquoi pas quelques graphiques illustratifs en utilisant Matplotlib...)